In [50]:
import pandas as pd
import numpy as np
import multiprocessing

multiprocessing.cpu_count()
import torch
import os
import pandas as pd
from torchvision.io import read_image
import numpy as np
import matplotlib.pyplot as plt
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, random_split
from torchvision import transforms
from torch import nn, optim
import torch.nn.functional as F
import torch.utils.data as data
import torch
import numpy as np
from collections import Counter
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from collections import Counter
from sklearn.metrics import roc_auc_score
from sklearn.metrics import confusion_matrix
from sklearn import metrics


np.random.seed(42)




In [21]:
# Trn_input

In [51]:
Trn_input = pd.read_pickle("/home/ffc4001/ukbiobankdata/Data/final_X_trainolink")
Test_input = pd.read_pickle("/home/ffc4001/ukbiobankdata/Data/final_X_testolink")
Trn_target = pd.read_pickle("/home/ffc4001/ukbiobankdata/Data/final_Y_trainolink")
Test_target = pd.read_pickle("/home/ffc4001/ukbiobankdata/Data/final_Y_testolink")
x_val =pd.read_pickle("/home/ffc4001/ukbiobankdata/Data/final_updated_X_valolink")
Y_val =pd.read_pickle("/home/ffc4001/ukbiobankdata/Data/final_updated_Y_valolink")

In [54]:
#DROP THE OLINK DATA FOR ABLATION TEST
Trn_input = Trn_input.iloc[:,:543]
Test_input = Test_input.iloc[:,:543]
x_val =x_val.iloc[:,:543]

In [55]:
Train_data = pd.concat([Trn_input, pd.DataFrame(Trn_target)], axis=1)
Test_data = pd.concat([Test_input, pd.DataFrame(Test_target)], axis=1)

In [56]:
val_data = pd.concat([x_val, pd.DataFrame(Y_val)], axis=1)

In [57]:
#Large Batch Size due to few cases, don't want batches with no cases
BATCH_SIZE_1 = 100000
BATCH_SIZE_2 = 100000

In [59]:
class VariantsDataset(Dataset):
    """Variants."""

    
    def __init__(self, data):
        self.data = torch.FloatTensor(data.values.astype('float'))
        print(self.data.shape)
        
    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        target = self.data[index][-1]
        data_val = self.data[index] [:-1]
        return data_val,target

In [60]:
# training and Test dataset 
train_dataset = VariantsDataset(Train_data)
test_dataset = VariantsDataset(Test_data)



torch.Size([26073, 544])
torch.Size([17335, 544])


In [61]:
kwargs = {'num_workers': 45, 'pin_memory': True} 

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE_1, shuffle=True, **kwargs)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE_2, shuffle=True, **kwargs)


In [62]:
train_loader, test_loader

(<torch.utils.data.dataloader.DataLoader at 0x7f1db3901a90>,
 <torch.utils.data.dataloader.DataLoader at 0x7f1db3aaae20>)

In [85]:
# Simple Neural network 
device ="cpu"
input_size = 543
hidden_size = 120
num_classes = 1
num_epochs = 200
learning_rate = 0.001
hidden_size2 = 40

In [87]:
#  ---------------  Model  ---------------
class LinearModel(nn.Module):
    def __init__(self, input_size, hidden_size, hidden_size2, num_classes):
        super(LinearModel, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.fc2 = nn.Linear(hidden_size, num_classes)
        self.fc3 = nn.Linear(hidden_size2, num_classes)
        self.lrelu1 = nn.LeakyReLU()
        self.lrelu2 = nn.LeakyReLU()
        self.dropout1 = nn.Dropout(0.25)
        self.dropout2 = nn.Dropout(0.25)

    def get_weights(self):
        return self.weight
    
    def forward(self,x):
        out = self.fc1(x)
        out = self.lrelu1(out)
        out = self.dropout1(out)
        out = self.fc2(out)
        #out = self.lrelu2(out)
        #out = self.dropout2(out)
        #out = self.fc3(out)
        return out

In [81]:
def train(model,device,train_loader,optimizer):
    model.train()
    correct = 0
    loss_total = 0
    y_true = []
    y_pred = []
    for i in train_loader:
        
        #LOADING THE DATA IN A BATCH
        data, target = i
        #print(target)
        #print(data)
        # moving the tensors to the configured device
        data, target = data.to(device), target.to(device)
       
        #FORWARD PASS
        target = target.float()
        output = model(data.float())
        #print(output)
        #print(target)
        loss = criterion(output, target.unsqueeze(1)) 
        
        loss_total += loss
        
        #BACKWARD AND OPTIMIZE
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        
        #PREDICTIONS BCELogitsloss()
        pred = np.round(torch.sigmoid(output.detach()))
        target = target.float()
        y_true.extend(target.tolist()) 
        y_pred.extend(pred.reshape(-1).tolist())
        

    print("AUC on training set is" , roc_auc_score(y_true,torch.sigmoid(output.detach())))

In [67]:
def test(model, device, test_loader):
    #model in eval mode skips Dropout etc
    model.eval()
    y_true = []
    y_pred = []
    
    # set the requires_grad flag to false as we are in the test mode
    with torch.no_grad():
        for data, target in test_loader:
            
            #LOAD THE DATA IN A BATCH
            #data,target = i
            #print(target)
            #print(data)
            # moving the tensors to the configured device
            data, target = data.to(device), target.to(device)
            
            
            output = model(data.float())
            
            #PREDICTIONS
            pred = np.round(torch.sigmoid(output))
            target = target.float()
            y_true.extend(target.tolist()) 
            y_pred.extend(pred.reshape(-1).tolist())
            
    #print(torch.sigmoid(output))        
    print("AUC on test set is" , roc_auc_score(y_true,torch.sigmoid(output)))
    #print("********************************************************")
    print(sum(y_true))
    print(sum(y_pred))
    cm=confusion_matrix(y_true,y_pred)
    tn, fp, fn, tp = cm.ravel()
    print("Sensitivity on test set is" , tp/(tp+fn))
    print("Specificity on test set is" , tn/(tn+fp)) 
    if epoch % 20 == 0:
        #roc_curve(y_test, classifier.predict_proba(x_test)[:,1])
        fpr, tpr, thresholds = metrics.roc_curve(y_true, torch.sigmoid(output))
        roc_auc = metrics.auc(fpr, tpr)
        display = metrics.RocCurveDisplay(fpr=fpr, tpr=tpr, roc_auc=roc_auc, estimator_name='Ridge Regression')
        display.plot()
        plt.show()



In [68]:
pos_weight  = torch.tensor(120)

In [88]:

# Loss and optimize
model = LinearModel(input_size, hidden_size, hidden_size2, num_classes).to(device)
criterion = nn.BCEWithLogitsLoss(pos_weight=pos_weight)
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate, weight_decay=1e-4)


In [1]:
for epoch in range(num_epochs):
        train(model,device,train_loader,optimizer)
        test(model,device,test_loader)

NameError: name 'num_epochs' is not defined

In [ ]:
batch1

In [ ]:
model(batch1)

In [ ]:
probabilites

In [ ]:
#AUC Curve, Sensitivity and specificity and F1

In [ ]:
#External validation

In [ ]:
#Validation

In [74]:
val_data = pd.concat([x_val, pd.DataFrame(Y_val)], axis=1)

In [75]:
val_dataset = VariantsDataset(val_data)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE_2, shuffle=True, **kwargs)

torch.Size([103, 544])


In [77]:
def val(model, device, val_loader):
    #model in eval mode skips Dropout etc
    model.eval()
    y_true = []
    y_pred = []
    
    # set the requires_grad flag to false as we are in the test mode
    with torch.no_grad():
        for data, target in val_loader:
            
            #LOAD THE DATA IN A BATCH
            #data,target = i
            #print(target)
            #print(data)
            # moving the tensors to the configured device
            data, target = data.to(device), target.to(device)
            
            
            output = model(data.float())
            
            #PREDICTIONS
            pred = np.round(torch.sigmoid(output))
            target = target.float()
            y_true.extend(target.tolist()) 
            y_pred.extend(pred.reshape(-1).tolist())
            
    #print(torch.sigmoid(output))        
    print("AUC on test set is" , roc_auc_score(y_true,torch.sigmoid(output)))
    #print("********************************************************")
    print(sum(y_true))
    print(sum(y_pred))
    cm=confusion_matrix(y_true,y_pred)
    tn, fp, fn, tp = cm.ravel()
    print("Sensitivity on test set is" , tp/(tp+fn))
    print("Specificity on test set is" , tn/(tn+fp)) 
    if epoch % 20 == 0:
        #roc_curve(y_test, classifier.predict_proba(x_test)[:,1])
        fpr, tpr, thresholds = metrics.roc_curve(y_true, torch.sigmoid(output))
        roc_auc = metrics.auc(fpr, tpr)
        display = metrics.RocCurveDisplay(fpr=fpr, tpr=tpr, roc_auc=roc_auc, estimator_name='Ridge Regression')
        display.plot()
        plt.show()

In [90]:
test(model,device,val_loader)

AUC on test set is 0.5094905094905096
26.0
35.0
Sensitivity on test set is 0.34615384615384615
Specificity on test set is 0.6623376623376623


In [ ]:
#Convert to HTML

In [79]:
#torch.save(model.state_dict(), "/home/ffc4001/ukbiobankdata/Data/finalvariantnnmodel1.pickle")
#torch.save(model, "/home/ffc4001/ukbiobankdata/Data/finalvariantnnmodel1.pickle")